In [1]:
import os
import time
import numpy as np
import copy
import torch
import torch.nn as nn
import pandas as pd

In [2]:
df = pd.read_csv('drive/MyDrive/energy_daily.csv')

In [3]:
df

,Date,W
0,2017-01-02,113884.376756
1,2017-01-03,115749.403766
2,2017-01-04,114738.968744
3,2017-01-05,113232.581490
4,2017-01-06,108208.275373
...,...,...
611,2018-09-30,101137.446320
612,2018-10-01,137820.726257
613,2018-10-02,123416.719997
614,2018-10-03,134916.629228


In [4]:
df=df['W']
print(df.max())
print(df.min())

264139.990829977
77824.75667106088


In [5]:
df = (df - df.min())/(df.max() - df.min()) #model normalizes

In [6]:
print(df)

0      0.193541
1      0.203551
2      0.198128
3      0.190043
4      0.163076
         ...   
611    0.125125
612    0.322013
613    0.244703
614    0.306426
615    0.322856
Name: W, Length: 616, dtype: float64


##### 30개씩 담아주기 (Monthly) - optional 

In [7]:
values = df.values

data = []
seq_len = 30
for i in range(len(values)-seq_len+1):
     #print(list(values[i:i+seq_len]))
     data.append(list(values[i:i+seq_len].reshape(-1)))
     if i==1:
         print(data)


df = pd.DataFrame(data)

print(df.info())

[[0.19354091063975745, 0.20355097244697024, 0.19812771746643917, 0.1900425640359459, 0.16307586891295162, 0.06904344583091582, 0.0349570923860602, 0.18012760270677525, 0.16469125395599923, 0.1487028859530828, 0.13743477929044826, 0.13301559374037872, 0.04964268971900893, 0.05406299437634862, 0.1419951607873467, 0.1985550586069767, 0.22165209785749193, 0.20904763565993031, 0.19524222575199546, 0.08167776950938133, 0.07355531421064455, 0.23949093765982468, 0.2645567593282403, 0.25473748957669723, 0.12338500831118433, 0.22040655473013002, 0.13140647904254613, 0.1355104752293138, 0.2633459623846408, 0.26805843375519817], [0.20355097244697024, 0.19812771746643917, 0.1900425640359459, 0.16307586891295162, 0.06904344583091582, 0.0349570923860602, 0.18012760270677525, 0.16469125395599923, 0.1487028859530828, 0.13743477929044826, 0.13301559374037872, 0.04964268971900893, 0.05406299437634862, 0.1419951607873467, 0.1985550586069767, 0.22165209785749193, 0.20904763565993031, 0.19524222575199546, 0

In [8]:
print(len(df))

587


##### Else (Daily ver.)

In [7]:
train_df = pd.DataFrame(df[:int(0.8*len(df))])
print(train_df)

            W
0    0.193541
1    0.203551
2    0.198128
3    0.190043
4    0.163076
..        ...
487  0.338293
488  0.341567
489  0.207684
490  0.196879
491  0.438719

[492 rows x 1 columns]


In [8]:
test_df = pd.DataFrame(df[int(0.8*len(df)):])
print(test_df)

            W
492  0.394864
493  0.380429
494  0.393407
495  0.412288
496  0.226514
..        ...
611  0.125125
612  0.322013
613  0.244703
614  0.306426
615  0.322856

[124 rows x 1 columns]


In [29]:
train = train_df.values.T
test = test_df.values.T
print(train.shape)
print(test.shape)

(1, 492)
(1, 124)


In [30]:
len(train)

1

Daily ; 14일 예측

In [31]:
val = train[:, -14:]
train = train[:, :-14]

In [32]:
print(train.shape)
print(val.shape)

(1, 478)
(1, 14)


In [33]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [34]:
class SeriesDataset(Dataset):

    def __init__(self, dataTrain, dataVal, dataTest, variable, chop_value, device):
     
        self.dataTrain = [torch.tensor(dataTrain[i]) for i in range(len(dataTrain))]  
        self.dataVal = [torch.tensor(dataVal[i]) for i in range(len(dataVal))]
        self.dataTest = [torch.tensor(dataTest[i]) for i in range(len(dataTest))]
        self.device = device
    
    def __len__(self):
        return len(self.dataTrain)

    def __getitem__(self, idx):
        return self.dataTrain[idx].to(self.device), \
                self.dataVal[idx].to(self.device), \
                self.dataTest[idx].to(self.device), \
                idx
                #self.dataInfoCat[idx].to(self.device), \

In [35]:
dataset = SeriesDataset(train, val, test, "Daily", 0 , 'cuda')

In [36]:
print(len(dataset.dataTrain[0]))
print(len(dataset.dataTrain))

478
1


In [37]:
print(len(dataset.dataVal[0]))
print(len(dataset.dataVal))

14
1


In [38]:
print(len(dataset.dataTest[0]))
print(len(dataset.dataTest))

124
1


In [39]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [40]:
for batch in dataloader:
    print(batch[0].shape)

torch.Size([1, 478])


In [41]:
len(dataset) #==len(self.dataTrain)

1

In [42]:
for grp_num in range(len(((1, 3), (7, 9)))):
    print(grp_num)

0
1


In [43]:
class ResidualDRNN(nn.Module):
    def __init__(self, config):
        super(ResidualDRNN, self).__init__()
        self.config = config

        layers = []
        for grp_num in range(len(self.config['dilations'])):     #0,1 두 번 반복 

            if grp_num == 0:
                input_size = 7   # daily = 7
            else:
                input_size = 50
            
            #input_size = self.config['input_size']
            
            l = DRNN(input_size,
                     self.config['state_hsize'],
                     n_layers=len(self.config['dilations'][grp_num]),
                     dilations=self.config['dilations'][grp_num],
                     cell_type='LSTM')

            layers.append(l)

        self.rnn_stack = nn.Sequential(*layers)